In [1]:
from glob import glob
from typing import List
from tqdm.notebook import tqdm
import pandas as pd
import os

In [2]:
def instance_list() -> List[str]:
    instances = list()
    instance_folder = os.path.join(os.path.abspath(''), '..', 'data', 'hop-tsiligirides')

    for instance in glob(os.path.join(instance_folder, '*.json')):
        instances.append(os.path.splitext(os.path.basename(instance))[0])

    return instances

In [4]:
algo_globs = [
    '*-nonlinear_concave_model-*',
    '*-nonlinear_concave_model_with_vi1_liftMTZ-*'
]

In [5]:
def read_results() -> pd.DataFrame:
    dfs = list()
    resfile_folder = os.path.join(os.path.abspath(''), '..', 'results', 'run')

    files = [glob(os.path.join(resfile_folder, glb)) for glb in algo_globs]
    files = [f for fs in files for f in fs]

    for resfile in tqdm(files):
        df = pd.read_csv(resfile)

        # Old results files were missing some header columns
        if 'add_vi1' not in df.columns:
            df['add_vi1'] = False
        if 'add_vi2' not in df.columns:
            df['add_vi2'] = False
        if 'add_vi3' not in df.columns:
            df['add_vi3'] = False
        if 'add_vi4' not in df.columns:
            df['add_vi4'] = False
        if 'liftMTZ' not in df.columns:
            df['liftMTZ'] = False

        # And some other column was named differently
        if 'actual_tour_obj' in df.columns:
            df.rename(columns=dict(actual_tour_obj='original_obj'), inplace=True)

        # Correct error in old results
        if df.loc[0, 'algorithm'] == 'nonlinear_model':
            df['algorithm'] = 'nonlinear_concave_model'

        dfs.append(df)

    return pd.concat(dfs)

In [6]:
def find_missing(instances: List[str], res: pd.DataFrame) -> List[str]:
    return [i for i in instances if len(res[res.instance_basename == i]) < 2]

In [7]:
instances = instance_list()

In [8]:
res = read_results()

  0%|          | 0/1568 [00:00<?, ?it/s]

In [9]:
missing = find_missing(instances=instances, res=res)

In [10]:
missing

[]

In [11]:
res.to_csv('baron-concave-bounds.csv', index=False)